<a href="https://colab.research.google.com/github/rakshitsakhuja/Signboard-Translation-from-Vernacular-Languages/blob/master/Text_Object_Detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive',force_remount=True)

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [ ]:
import os
os.chdir('/content/drive/My Drive/Text Detection')

In [ ]:
!/opt/bin/nvidia-smi

Thu Jul  2 04:58:45 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 418.67       Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   34C    P8     9W /  70W |      0MiB / 15079MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
                                                                               
+-----------------------------------------------------------------------------+
| Processes:                                                       GPU Memory |
|  GPU  

In [ ]:
# !tar -xvf '/content/drive/My Drive/Text Detection/Datasets/real_Image_dataset_Detection.tar'

In [ ]:
# !rm Datasets/real_Image_dataset_Detection/Annotation/46.\ videoblocks-india-tourism-delhi-sign-hindi-and-english-nighttime-new-delhi-india_rx3i43x6g_thumbnail-full01.txt
# !rm Datasets/real_Image_dataset_Detection/Annotation/270.\ palika-bazaar.txt
# !rm Datasets/real_Image_dataset_Detection/Image/46.\ videoblocks-india-tourism-delhi-sign-hindi-and-english-nighttime-new-delhi-india_rx3i43x6g_thumbnail-full01.jpg
# !rm Datasets/real_Image_dataset_Detection/Image/270.\ palika-bazaar.jpg

In [ ]:
import glob
import os
from os.path import basename

import torch

from checkpoint import load_ckp
from config import parameters
from dataloader_factory import get_custom_dataset
from models import pretrained_model
from train import train_fn


def main():
    os.makedirs('/'.join(parameters['checkpoint_path'].split('/')[:-1]), exist_ok=True)
    os.makedirs('/'.join(parameters['best_model_path'].split('/')[:-1]), exist_ok=True)


    device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
    print(f'Device : {device} is selected')
    file_names = [''.join(basename(i)[:-4]) for i in
                  glob.glob(os.path.join(parameters['image_dir'], '*'))]
    # print(file_names)
    train_loader, validation_loader = get_custom_dataset(file_names, parameters['train_frac'])

    model = pretrained_model('fasterrcnn_resnet50_fpn')
    model = model.to(device)

    params = [p for p in model.parameters() if p.requires_grad]
    optimizer = torch.optim.SGD(params, lr=parameters['learning_rate'],
                                momentum=parameters['momentum'],
                                weight_decay=parameters['weight_decay'])
    try:
        model, optimizer, start_epoch = load_ckp(parameters['checkpoint_path'], model, optimizer)
    except Exception:
        print('No Previous Checkpoint Found....Training the model from scratch')
        start_epoch = 0
    best_loss = 1e10
    train_fn(start_epoch, parameters['epoch'], train_loader, validation_loader, model, device, optimizer, best_loss,parameters['checkpoint_path'],parameters['best_model_path'])


   

Downloading: "https://download.pytorch.org/models/fasterrcnn_resnet50_fpn_coco-258fb6c6.pth" to /root/.cache/torch/checkpoints/fasterrcnn_resnet50_fpn_coco-258fb6c6.pth


In [ ]:
main()

Device : cuda is selected
Transformation Done
426
Starting Training


/usr/local/lib/python3.6/dist-packages/torch/nn/functional.py:2854: UserWarning: The default behavior for interpolate/upsample with float scale_factor will change in 1.6.0 to align with other frameworks/libraries, and use scale_factor directly, instead of relying on the computed output size. If you wish to keep the old behavior, please set recompute_scale_factor=True. See the documentation of nn.Upsample for details. 
  warnings.warn("The default behavior for interpolate/upsample with float scale_factor will change "
/pytorch/torch/csrc/utils/python_arg_parser.cpp:756: UserWarning: This overload of nonzero is deprecated:
	nonzero(Tensor input, *, Tensor out)
Consider using one of the following signatures instead:
	nonzero(Tensor input, *, bool as_tuple)


Iteration #10 loss: 0.1886848360300064
Iteration #20 loss: 0.07751213014125824
Iteration #30 loss: nan
Iteration #40 loss: 0.2984048128128052
Iteration #50 loss: 0.1489311307668686
Iteration #60 loss: 0.17469733953475952
Iteration #70 loss: 2.86879301071167
Iteration #80 loss: 0.21998392045497894
Iteration #90 loss: 2.240328073501587
Epoch #23 Train loss: nan, Validation Loss : Commented
Iteration #100 loss: 0.16861438751220703
Iteration #110 loss: nan
Iteration #120 loss: 0.13263435661792755
Iteration #130 loss: 0.19540539383888245
Iteration #140 loss: 0.11648991703987122
Iteration #150 loss: 5.731350898742676
Iteration #160 loss: 0.1478748619556427
Iteration #170 loss: 0.17970308661460876
Iteration #180 loss: 0.35047447681427
Iteration #190 loss: 0.12469891458749771
Epoch #24 Train loss: nan, Validation Loss : Commented
Iteration #200 loss: 0.11974459886550903
Iteration #210 loss: 0.14871729910373688
Iteration #220 loss: 0.08353939652442932
Iteration #230 loss: 0.12854193150997162
It

In [ ]:
parameters['image_dir']

In [ ]:
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
print(f'Device : {device} is selected')
file_names = [''.join(basename(i)[:-4]) for i in
                  glob.glob(os.path.join(parameters['image_dir'], '*'))]
    # print(file_names)
train_loader, validation_loader = get_custom_dataset(file_names, parameters['train_frac'])

model = pretrained_model('fasterrcnn_resnet50_fpn')
model = model.to(device)

params = [p for p in model.parameters() if p.requires_grad]
optimizer = torch.optim.SGD(params, lr=parameters['learning_rate'],
                                momentum=parameters['momentum'],
                                weight_decay=parameters['weight_decay'])

Device : cuda is selected
Transformation Done
426


In [ ]:
load_ckp(checkpoint_fpath = parameters['checkpoint_path'], model=model, optimizer = optimizer)

(FasterRCNN(
   (transform): GeneralizedRCNNTransform(
       Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
       Resize(min_size=(800,), max_size=1333, mode='bilinear')
   )
   (backbone): BackboneWithFPN(
     (body): IntermediateLayerGetter(
       (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
       (bn1): FrozenBatchNorm2d()
       (relu): ReLU(inplace=True)
       (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
       (layer1): Sequential(
         (0): Bottleneck(
           (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
           (bn1): FrozenBatchNorm2d()
           (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
           (bn2): FrozenBatchNorm2d()
           (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
           (bn3): FrozenBatchNorm2d()
           (relu): ReLU(inplace=True)
           

In [ ]:
model

NameError: ignored